# Workbook 03 - Quick Start with Documents Store

Documents in GDN are JSON objects. These objects can be nested (to any depth) and may contain lists. Each document has a unique primary key which identifies it within its collection. Furthermore, each document is uniquely identified by its document handle across all collections. Different revisions of the same document (identified by its handle) can be distinguished by their document revision. Any transaction only ever sees a single revision of a document.

For example:

In [ ]:
{
  "_id" : "myusers/3456789",
  "_key" : "3456789",
  "_rev" : "14253647",
  "firstName" : "John",
  "lastName" : "Doe",
  "address" : {
    "street" : "Road To Nowhere 1",
    "city" : "Gotham"
  },
  "hobbies" : [
    {name: "swimming", howFavorite: 10},
    {name: "biking", howFavorite: 6},
    {name: "programming", howFavorite: 4}
  ]
}

All documents contain special attributes:

- the document handle is stored as a string in _id,
- the document's primary key in _key and
- the document revision in _rev.

The value of the _key attribute can be specified by the user when creating a document. _id and _key values are immutable once the document has been created. The _rev value is maintained by GDN automatically.

## Pre-requisite

Lets Assume 
- you have already made a tenant account, and have a username and password
- you have installed the jsc8 drivers as explained in section 01
- you have generated an API Key as explained in section 01

In [ ]:
#/* run this once to install javascript kernal and jsc8 in google colab, then reload, and then skip this
!npm install jsc8
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global  # */

## Step 03-A - Connect to GDN

In [14]:
const jsc8 = require("jsc8");

// Simple Way
const client = new jsc8({url: "https://gdn.paas.macrometa.io", token: "XXXX", fabricName: '_system'});
// ----- OR -----
// const client = new jsc8({url: "https://gdn.paas.macrometa.io", apiKey: "XXXX", fabricName: '_system'});

// To use advanced options
// const client = new jsc8("https://gdn.paas.macrometa.io"); 
// client.login(email, password).then(result => console.log("Login successfully", result)).catch(err => console.error("Error while login", err.message))

## Step 03-B - Get GeoFabric Details

To get details of fabric,

In [8]:
// you might not need this, but if you wanted to select a
// specific GeoFabric you can find out whats available by executing this code!

async function getGeoFabricMetaData() {
  console.log("Get geo fabric details...");

  try {
    const fabrics = await client.get();
    console.log(fabrics);
  } catch (err) {
    console.log("Error while fetching GeoFabric details", err.message);
  }
}

getGeoFabricMetaData();


## Step 03-C - Create Collection

We can now create collection in the fabric. To do this, first you connect to fabric and then create a collection called employees.

The below example shows the steps.

In [16]:
const collection_name = "employees";

async function createCollection() {
  let collectionDetails;
  try {
    // Create a new collection if it does not exist
    if (await client.hasCollection(collection_name)) {
      console.log("Collection exists with name ", collection_name);
      return;
    }
    collectionDetails = await client.createCollection(collection_name);
    console.log("Collection created! ", collection_name);
  } catch (e) {
    return "Collection creation did not succeed due to " + e.message;
  }
}

createCollection();

## Step 03-D - Create Index

Let's add a hash_index called emails to our collection employees. Please refer to reference guide for details on other available index types.

In [24]:
async function addHashIndex() {
  console.log("Add Hash Index");
  try {
    const hashIndex = await client.addHashIndex(
      collection_name,
      (fields = ["firstname", "lastname"]),
      { unique: true }
    );
    console.log("Index created: ", hashIndex);
  } catch (err) {
    console.log("Error while adding Hash Index", err.message);
  }
}

addHashIndex();


## Step 03-E - Insert Documents

Let's insert documents to the employees collection as shown below.

In [25]:
let docs = [
  {
    _key: "James",
    firstname: "James",
    lastname: "Kirk",
    email: "[email protected]",
  },
  {
    _key: "Han",
    firstname: "Han",
    lastname: "Solo",
    email: "[email protected]",
  },
  {
    _key: "Bruce",
    firstname: "Bruce",
    lastname: "Wayne",
    email: "[email protected]",
  },
];

async function insertDocument() {
  return await client.insertDocument(collection_name, docs);
}

insertDocument()
  .then((result) => console.log("Documents inserted"))
  .catch((err) => console.log("Error while inserting documents", err.message));

## Step 03-F - Query documents using C8QL

8QL is C8's query language. You can execute C8QL query on our newly created collection employees to get its contents.

The query "FOR employee IN employees RETURN employee" is equivalent to SQL's SELECT query.

In [26]:
async function c8Queries() {
  let docs = await client.executeQuery(
    "FOR employee IN employees RETURN employee"
  );
  console.log(docs);
}

c8Queries();

## Step 03-G - Get realtime updates

Example for real-time updates from a collection in fabric:

Note: for this to work you need to enable Stream from the Macrometa Dashboard. Do this by opening up the dashboard, select collections, find the collection named "employees" and select "enable Stream"

Now run the cell below, return to the Macrometa Dashboard and modify and save fields in the collection. you should see the real time updates below as soon as you hit save!. 

the websocket will time out after 60sec if no updates are seen, so you might need to re run the cell code if you see a timed out message below. :)

In [28]:
async function callback_fn(collection) {
  await console.log("Connection open on ", collection.name);
}

async function realTimeListener() {
  const listener = await client.onCollectionChange(collection_name);

  listener.on("message", (msg) => console.log("message=>", msg));
  listener.on("open", () => {
    this.callback_fn(collection);
  });
  listener.on("close", () => console.log("connection closed"));
}

realTimeListener();

## Section Completed!

TBC...